In [1]:
import pandas as pd 
import numpy as np 
from scipy.stats import entropy
import matplotlib.pyplot as plt

from engine_hms_trainer import *
from engine_hms_model import JobConfig, ModelConfig

import torch
from torch import nn
import torch.nn.functional as F

import warnings
warnings.filterwarnings('ignore')

/home/shiyi/miniconda3/envs/kaggle/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/shiyi/miniconda3/envs/kaggle/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
seed_everything(JobConfig.SEED)

## Config EfficientNet 
# ModelConfig.EPOCHS = 6
# ModelConfig.MODEL_BACKBONE = 'tf_efficientnet_b2'
# ModelConfig.MODEL_NAME = "ENet_b2_mlp"
# ModelConfig.AUGMENT = True
# ModelConfig.USE_KAGGLE_SPECTROGRAMS = True
# ModelConfig.USE_EEG_SPECTROGRAMS = True
# ModelConfig.REGULARIZATION = None
# ModelConfig.AUGMENTATIONS = ['xy_masking']

## Config ViTMAE
ModelConfig.EPOCHS = 6
ModelConfig.MODEL_BACKBONE = 'vit_mae_base'
ModelConfig.MODEL_NAME = "ViTMAE_base"
ModelConfig.AUGMENT = True
ModelConfig.USE_KAGGLE_SPECTROGRAMS = True
ModelConfig.USE_EEG_SPECTROGRAMS = True
ModelConfig.REGULARIZATION = None
ModelConfig.AUGMENTATIONS = ['xy_masking']
ModelConfig.MAE_PRETRAINED_WEIGHTS = "./outputs/vit_mae_pretraining/best_mae_model.pth"

hms_predictor = HMSPredictor(JobConfig, ModelConfig)

****************************************************************************************************
Script Start: Tue Mar 12 21:26:25 2024
Model Configurations:
MODEL_NAME: ViTMAE_base
MODEL_BACKBONE: vit_mae_base
BATCH_SIZE: 16
EPOCHS: 6
GRADIENT_ACCUMULATION_STEPS: 2
DROP_RATE: 0.15
DROP_PATH_RATE: 0.25
WEIGHT_DECAY: 0.01
REGULARIZATION: None
USE_KAGGLE_SPECTROGRAMS: True
USE_EEG_SPECTROGRAMS: True
AMP: True
AUGMENT: True
AUGMENTATIONS: ['xy_masking']
PRINT_FREQ: 50
FREEZE: False
NUM_FROZEN_LAYERS: 0
NUM_WORKERS: 0
MAX_GRAD_NORM: 10000000.0
MAE_PRETRAINED_WEIGHTS: ./outputs/vit_mae_pretraining/best_mae_model.pth
MAE_HIDDEN_DROPOUT_PROB: 0.05
MAE_ATTENTION_DROPOUT_PROB: 0.05
****************************************************************************************************


In [3]:
train_easy, train_hard, all_specs, all_eegs = hms_predictor.load_train_data()

print(train_easy.shape)
print(train_hard.shape)

# check if contain NaN
print(train_easy.isnull().sum().sum())
print(train_hard.isnull().sum().sum())

display(train_easy.head())
print(" ")
display(train_hard.head())

(13996, 12)
(6187, 12)
0
0


,eeg_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,spectrogram_id,min,max,patient_id,target
0,642382,0.0,0.0,0.0,0.0,0.0,1.0,14960202,1008.0,1032.0,5955,Other
1,751790,0.0,0.0,1.0,0.0,0.0,0.0,618728447,908.0,908.0,38549,GPD
2,778705,0.0,0.0,0.0,0.0,0.0,1.0,52296320,0.0,0.0,40955,Other
3,1629671,1.0,0.0,0.0,0.0,0.0,0.0,2036345030,0.0,160.0,37481,Seizure
4,2061593,0.0,0.0,0.0,0.0,0.0,1.0,320962633,1450.0,1450.0,23828,Other


,eeg_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,spectrogram_id,min,max,patient_id,target
0,568657,0.000000,0.000000,0.250000,0.000000,0.166667,0.583333,789577333,0.0,16.0,20654,Other
1,582999,0.000000,0.857143,0.000000,0.071429,0.000000,0.071429,1552638400,0.0,38.0,20230,LPD
2,1895581,0.076923,0.000000,0.000000,0.000000,0.076923,0.846154,128369999,1138.0,1138.0,47999,Other
3,2482631,0.000000,0.000000,0.133333,0.066667,0.133333,0.666667,978166025,1902.0,1944.0,20606,Other
4,2521897,0.000000,0.000000,0.083333,0.083333,0.333333,0.500000,673742515,0.0,4.0,62117,Other


In [4]:
hms_predictor.train_folds(train_easy, train_hard, all_specs, all_eegs)

Fold: 0 || Valid size 6867 
- First Stage 


Train:   0%|          | 0/566 [00:00<?, ?batch/s]

Epoch: [1][0/566]Elapsed 0.82s | Loss: 0.8747 Grad: inf LR: 4.0000e-06
Epoch: [1][50/566]Elapsed 3.97s | Loss: 0.7918 Grad: 22935.7832 LR: 5.2855e-06
Epoch: [1][100/566]Elapsed 7.05s | Loss: 0.7631 Grad: 24325.8926 LR: 9.0731e-06
Epoch: [1][150/566]Elapsed 10.14s | Loss: 0.7482 Grad: 22231.9473 LR: 1.5160e-05
Epoch: [1][200/566]Elapsed 13.23s | Loss: 0.7361 Grad: 26599.8477 LR: 2.3220e-05
Epoch: [1][250/566]Elapsed 16.31s | Loss: 0.7207 Grad: 42741.4062 LR: 3.2822e-05
Epoch: [1][300/566]Elapsed 19.39s | Loss: 0.7094 Grad: 32611.2246 LR: 4.3451e-05
Epoch: [1][350/566]Elapsed 22.47s | Loss: 0.6982 Grad: 73515.8047 LR: 5.4537e-05
Epoch: [1][400/566]Elapsed 25.55s | Loss: 0.6836 Grad: 39497.6602 LR: 6.5488e-05
Epoch: [1][450/566]Elapsed 28.65s | Loss: 0.6715 Grad: 40204.4180 LR: 7.5717e-05
Epoch: [1][500/566]Elapsed 31.74s | Loss: 0.6606 Grad: 35129.3281 LR: 8.4675e-05
Epoch: [1][550/566]Elapsed 34.83s | Loss: 0.6480 Grad: 37107.0977 LR: 9.1883e-05
Epoch: [1][565/566]Elapsed 35.78s | Loss:

Valid:   0%|          | 0/430 [00:00<?, ?batch/s]

Epoch: [1][0/430]Elapsed 0.05s | Loss: 0.6996
Epoch: [1][50/430]Elapsed 2.45s | Loss: 0.4806
Epoch: [1][100/430]Elapsed 4.93s | Loss: 0.4930
Epoch: [1][150/430]Elapsed 7.52s | Loss: 0.4981
Epoch: [1][200/430]Elapsed 10.16s | Loss: 0.5050
Epoch: [1][250/430]Elapsed 12.80s | Loss: 0.5113
Epoch: [1][300/430]Elapsed 15.47s | Loss: 0.5092
Epoch: [1][350/430]Elapsed 18.14s | Loss: 0.4909
Epoch: [1][400/430]Elapsed 20.79s | Loss: 0.4790


...

Train:   0%|          | 0/327 [00:00<?, ?batch/s]

Epoch: [6][0/327]Elapsed 0.07s | Loss: 0.1783 Grad: 113428.1484 LR: 7.2570e-05
Epoch: [6][50/327]Elapsed 3.51s | Loss: 0.1704 Grad: 77080.4375 LR: 7.0565e-05
Epoch: [6][100/327]Elapsed 6.83s | Loss: 0.1686 Grad: 58343.8945 LR: 6.8518e-05
Epoch: [6][150/327]Elapsed 10.16s | Loss: 0.1642 Grad: 89405.5078 LR: 6.6436e-05
Epoch: [6][200/327]Elapsed 13.39s | Loss: 0.1643 Grad: 68858.6875 LR: 6.4320e-05
Epoch: [6][250/327]Elapsed 16.57s | Loss: 0.1661 Grad: 83269.4219 LR: 6.2177e-05
Epoch: [6][300/327]Elapsed 19.86s | Loss: 0.1666 Grad: 47871.1562 LR: 6.0009e-05
Epoch: [6][326/327]Elapsed 21.55s | Loss: 0.1659 Grad: 77176.4609 LR: 5.8874e-05


Valid:   0%|          | 0/172 [00:00<?, ?batch/s]

Epoch: [6][0/172]Elapsed 0.05s | Loss: 0.4316
Epoch: [6][50/172]Elapsed 2.51s | Loss: 0.5050
Epoch: [6][100/172]Elapsed 5.07s | Loss: 0.4964
Epoch: [6][150/172]Elapsed 7.73s | Loss: 0.4309


----------------------------------------------------------------------------------------------------
Epoch 6 - Average Train Loss: 0.1659 | Average Valid Loss: 0.4039 | Time: 30.52s
Best model found in epoch 6 | valid loss: 0.4039
Fold 4 Valid Loss: (Easy) 1.0062 | (Hard) 0.4251
Elapse: 3.02 min 
Training Complete!
CV Result (Stage=1): 0.7553410530090332
CV Result (Stage=2): 0.7453713417053223
Elapse: 46.20 min 


In [ ]:
pd.set_option('display.max_columns', None)
KL_CRITERION = nn.KLDivLoss(reduction='batchmean')
SOFTMAX = nn.Softmax(dim=1)

TARGET2ID = {
    'Seizure': 0,
    'LPD': 1,
    'GPD': 2,
    'LRDA': 3,
    'GRDA': 4,
    'Other': 5
}

from kl_divergence import score as kaggle_score 

def calc_kaggle_score(solution, submission):
    solution = solution.to_frame().T
    solution[TARGETS] = solution[TARGETS].astype(np.float32)
    submission = submission.to_frame().T
    submission.columns = ['eeg_id'] + TARGETS
    submission[TARGETS] = submission[TARGETS].astype(np.float32)
    
    return kaggle_score(solution, submission, 'eeg_id')


def calc_kl_div(p, q):
    p = torch.tensor(p.astype(np.float32)).unsqueeze(0)
    q = torch.tensor(q.astype(np.float32)).unsqueeze(0)
    kl_score = KL_CRITERION(F.log_softmax(p, dim=1), q)
    return kl_score.item()

In [ ]:
# csv_path = './outputs/ENet_b2_softmax/ENet_b2_softmax_oof_2.csv'

oof_df = pd.read_csv('./outputs/ViTMAE_base/ViTMAE_base_oof_2.csv')
oof_df['target_pred'] = oof_df[TARGETS_PRED].apply(lambda x: np.argmax(x), axis=1)
oof_df['target_id'] = oof_df['target'].map(TARGET2ID)

oof_df["kl_loss"] = oof_df.apply(
    lambda row: 
        KL_CRITERION(
            F.log_softmax(
                SOFTMAX(
                    torch.tensor(row[TARGETS_PRED].astype(np.float32)).unsqueeze(0)
                    )
                ), 
            torch.tensor(row[TARGETS].astype(np.float32))
            ).numpy(),
    axis=1)

oof_df["kl_loss"] = oof_df['kl_loss'].astype(np.float32)

oof_df[TARGETS_PRED] = SOFTMAX( torch.tensor(oof_df[TARGETS_PRED].values.astype(np.float32)))

oof_df.head()

In [ ]:
oof_df["kl_loss"].plot(kind='hist', bins=100, title='KL Loss Distribution', figsize=(8, 5))
print(oof_df["kl_loss"].mean())
plt.show()

In [ ]:
plot_oof = oof_df.copy()

# plot confusion matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(plot_oof['target_id'], plot_oof['target_pred']) # (y_true, y_pred)
cm = cm / cm.sum(axis=1)[:, np.newaxis]

fig = plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, cmap='Blues', xticklabels=TARGET2ID.keys(), yticklabels=TARGET2ID.keys())
plt.xlabel('Predicted', fontsize=12)
plt.ylabel('True', fontsize=12)
plt.show()

In [ ]:
# new figure
fig, axes = plt.subplots(6, 5, figsize=(18, 16), sharex=True, sharey=True)

plot_oof = oof_df[oof_df['kl_loss'] > 0.2]

for row in range(axes.shape[0]):
    row_selects = plot_oof[plot_oof['target_id']==row]
    target_label = BRAIN_ACTIVITY[row]
    for col in range(axes.shape[1]):
        ax = axes[row, col]
        idx = np.random.choice(row_selects.index)
        df_rows = plot_oof.loc[idx]
        ax.plot(df_rows[TARGETS].values , label='True')
        ax.plot(df_rows[TARGETS_PRED].values, label='Pred')
        ax.set_title(f"{idx} | KL: {df_rows['kl_loss']:.4f} ") #
        ax.set_xticks(range(6))
        ax.set_xticklabels(BRAIN_ACTIVITY)
        ax.grid(True)
        ax.legend()
        if col == 0:
            ax.set_ylabel(target_label, fontsize=12)
       
fig.tight_layout()
plt.show()

In [ ]:
csv_path = './outputs/ENet_b2_xymasking_remove_less/ENet_b2_xymasking_remove_less_oof_2.csv'

oof_df = pd.read_csv(csv_path)

oof_df["kl_loss"] = oof_df.apply(
    lambda row: 
        KL_CRITERION(
            F.log_softmax(
                SOFTMAX(
                    torch.tensor(row[TARGETS_PRED].astype(np.float32)).unsqueeze(0)
                    )
                ), 
            torch.tensor(row[TARGETS].astype(np.float32))
            ).numpy(),
    axis=1)

oof_df["kl_loss"] = oof_df['kl_loss'].astype(np.float32)

# y_pred = oof_df[TARGETS_PRED].values.astype(np.float32)
# y_pred_smax = SOFTMAX(torch.tensor(y_pred)).numpy()
# oof_df[TARGETS_PRED] = y_pred_smax

oof_df['target_pred'] = oof_df[TARGETS_PRED].apply(lambda x: np.argmax(x), axis=1)
oof_df['target_id'] = oof_df['target'].map(TARGET2ID)

oof_df.head()

In [ ]:
oof_df["kl_loss"].plot(kind='hist', bins=100, title='KL Loss Distribution', figsize=(8, 5))
print(oof_df["kl_loss"].mean())
plt.show()

In [ ]:
KL_CRITERION(
    torch.log(
        SOFTMAX(
            torch.tensor(oof_df[TARGETS_PRED].values.astype(np.float32))
            )
        ), 
    torch.tensor(oof_df[TARGETS].values.astype(np.float32))
    ).numpy(),

In [ ]:
submission_df = oof_df[['eeg_id']+TARGETS_PRED].copy()
submission_df.columns = ['eeg_id'] + TARGETS

solution_df = oof_df[['eeg_id']+TARGETS].copy()

score_value = kaggle_score(solution_df, submission_df, 'eeg_id')

score_value

In [ ]:
plot_oof = oof_df.copy()

# plot confusion matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(plot_oof['target_id'], plot_oof['target_pred']) # (y_true, y_pred)
cm = cm / cm.sum(axis=1)[:, np.newaxis]

fig = plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, cmap='Blues', xticklabels=TARGET2ID.keys(), yticklabels=TARGET2ID.keys())
plt.xlabel('Predicted', fontsize=12)
plt.ylabel('True', fontsize=12)
plt.show()

In [ ]:
# new figure
fig, axes = plt.subplots(6, 5, figsize=(18, 16), sharex=True, sharey=True)

plot_oof = oof_df[oof_df['kl_loss'] > 0.2]

for row in range(axes.shape[0]):
    row_selects = plot_oof[plot_oof['target_id']==row]
    target_label = BRAIN_ACTIVITY[row]
    for col in range(axes.shape[1]):
        ax = axes[row, col]
        idx = np.random.choice(row_selects.index)
        df_rows = plot_oof.loc[idx]
        ax.plot(df_rows[TARGETS].values , label='True')
        ax.plot(df_rows[TARGETS_PRED].values, label='Pred')
        ax.set_title(f"{idx} | KL: {df_rows['kl_loss']:.4f} ") #
        ax.set_xticks(range(6))
        ax.set_xticklabels(BRAIN_ACTIVITY)
        ax.grid(True)
        ax.legend()
        if col == 0:
            ax.set_ylabel(target_label, fontsize=12)
       
fig.tight_layout()
plt.show()




In [ ]:
oof_df[oof_df['eeg_id'] == 11127485]

In [ ]:
oof_df[oof_df['eeg_id'].duplicated()] #oof_df.shape #.groupby('eeg_id')['patient_id'].agg(['nunique', 'count']).sort_values(by='count', ascending=False).head(10)

In [ ]:
score_kaggle = oof_df2.loc[:10].apply(lambda row: calc_kaggle_score(row[['eeg_id']+TARGETS], row[['eeg_id']+TARGETS_PRED]), axis=1)
score_kaggle

In [ ]:
submission_df = oof_df2[['eeg_id']+TARGETS_PRED].copy()
submission_df.columns = ['eeg_id'] + TARGETS

solution_df = oof_df2[['eeg_id']+TARGETS].copy()

score_value = kaggle_score(solution_df, submission_df, 'eeg_id')

score_value

In [ ]:
# oof_df1, cv_1 = analyze_oof("./outputs/ENet_b2_xymasking_remove_less/ENet_b2_xymasking_remove_less_oof_1.csv")
# print(cv_1)
# oof_df1.head()